In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 34.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=030f2f11e554349eaa2a487968e60d8cd33cd73f748fd4ba037f3db0a8cd6e84
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


# Создаём точку входа в Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [ ]:
spark

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Можно использовать только rdd.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [ ]:
train = sc.textFile('/content/train.csv')
test = sc.textFile('/content/test.csv')

In [ ]:
# Выведите содержимое каждого файла (часть строк) (1 балл)
###############
data = sc.textFile('train.csv')
data.take(3)

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2']

In [ ]:
data = sc.textFile('test.csv')
data.take(3)

['id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi',
 '1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0',
 '2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0']

In [ ]:
###############

In [ ]:
data_header = data.first()
data_header

'id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi'

In [ ]:
data = data.filter(lambda row: row != data_header)

In [ ]:
data.take(3)

['1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0',
 '2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0',
 '3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1']

##  Преобразуем train и test

In [ ]:
train_first_row = train.first()

train = train\
    .filter(lambda row: row != train_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')])

In [ ]:
beautiful_print(train, 3, 'Our rdd')

Our rdd
	 [842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0, 1.0]
	 [1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0, 2.0]
	 [563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0, 2.0]


In [ ]:
# Преобразуйте test (1 балл)
###############
test_first_row = test.first()

test = test\
    .filter(lambda row: row != test_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')])

In [ ]:
beautiful_print(test, 3, 'Our rdd')

Our rdd
	 [1.0, 1043.0, 1.0, 1.8, 1.0, 14.0, 0.0, 5.0, 0.1, 193.0, 3.0, 16.0, 226.0, 1412.0, 3476.0, 12.0, 7.0, 2.0, 0.0, 1.0, 0.0]
	 [2.0, 841.0, 1.0, 0.5, 1.0, 4.0, 1.0, 61.0, 0.8, 191.0, 5.0, 12.0, 746.0, 857.0, 3895.0, 6.0, 0.0, 7.0, 1.0, 0.0, 0.0]
	 [3.0, 1807.0, 1.0, 2.8, 0.0, 1.0, 0.0, 27.0, 0.9, 186.0, 3.0, 4.0, 1270.0, 1366.0, 2396.0, 17.0, 10.0, 10.0, 0.0, 1.0, 1.0]


In [ ]:
###############

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.html#rdd-apis)

PS: нужно сделать средсвтвами rdd pd.concat([train, test,], axis=0)

In [ ]:
# Объедините train и test (2 балла)
###############
# Ваш код здесь
# data = rdd pd.concat([train, test,], axis=0)
###############

In [ ]:
data.take(3)

['1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0',
 '2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0',
 '3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1']

In [ ]:
RDD.intersection()

## Рассчитайте соотношение сторон телефона и экрана

In [ ]:
# Выведите отсортированное распределение соотношений сторон экрана(1 балла)
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)
###############
col_num = data_header.split(',').index('sc_w')
data.map(lambda row: (row[col_num], 1)).reduceByKey(lambda x, y: x + y ).sortByKey().collect()
###############

[(',', 617),
 ('0', 75),
 ('1', 124),
 ('2', 42),
 ('3', 30),
 ('4', 28),
 ('5', 28),
 ('6', 21),
 ('7', 15),
 ('8', 10),
 ('9', 10)]

In [ ]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# точек на дюйм
###############
col_num = data_header.split(',').index('px_width')
data.map(lambda row: (row[col_num], 1)).reduceByKey(lambda x, y: x + y ).sortByKey().collect()
###############

[(',', 376),
 ('.', 1),
 ('0', 39),
 ('1', 82),
 ('2', 34),
 ('3', 37),
 ('4', 38),
 ('5', 159),
 ('6', 66),
 ('7', 48),
 ('8', 63),
 ('9', 57)]

# JOIN
Повторите вышеописанное задание с помощью одной из функций ниже (отдельно рассчитайте для train и test, затем объедините результат)

Пример для двух RDDs: (rdd = {(1, 2), (3, 4), (3, 6)} other = {(3, 9)})

Имя функции |	Purpose |	Example |	Result
------------- |	------- |	------- |	------
subtractByKey |Remove elements with a key present in the other RDD.| rdd.subtractByKey(other) | {(1, 2)}
join | Perform an inner join between two RDDs. | rdd.join(other) | {(3, (4, 9)), (3, (6, 9))}
rightOuterJoin | Perform a join between two RDDs where the key must be present in the first RDD. | rdd.rightOuterJoin(other) | {(3,(Some(4),9)), (3,(Some(6),9))}leftOuterJoin | Perform a join between two RDDs where the key must be present in the other RDD. | rdd.leftOuterJoin(other) | {(1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9)))}
cogroup | Group data from both RDDs sharing the same key. | rdd.cogroup(other) | {(1,([2],[])), (3,([4, 6],[9]))}


In [ ]:
# 2 балла
###############
# Ваш код здесь
###############

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

In [ ]:
df = spark.read.csv('train.csv', header=True)
df.show(1, vertical=True)

-RECORD 0-------------
 battery_power | 842  
 blue          | 0    
 clock_speed   | 2.2  
 dual_sim      | 0    
 fc            | 1    
 four_g        | 0    
 int_memory    | 7    
 m_dep         | 0.6  
 mobile_wt     | 188  
 n_cores       | 2    
 pc            | 2    
 px_height     | 20   
 px_width      | 756  
 ram           | 2549 
 sc_h          | 9    
 sc_w          | 7    
 talk_time     | 19   
 three_g       | 0    
 touch_screen  | 0    
 wifi          | 1    
 price_range   | 1    
only showing top 1 row



In [ ]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|    battery_power|              blue|       clock_speed|          dual_sim|               fc|             four_g|        int_memory|             m_dep|        mobile_wt|           n_cores|               pc|         px_height|         px_width|               ram|             sc_h|             sc_w|        talk_time|          three_g|      touch_screen|              wifi|       price_range|
+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-

In [ ]:
df = spark.read.csv('test.csv', header=True)
df.show(1, vertical=True)

-RECORD 0-------------
 id            | 1    
 battery_power | 1043 
 blue          | 1    
 clock_speed   | 1.8  
 dual_sim      | 1    
 fc            | 14   
 four_g        | 0    
 int_memory    | 5    
 m_dep         | 0.1  
 mobile_wt     | 193  
 n_cores       | 3    
 pc            | 16   
 px_height     | 226  
 px_width      | 1412 
 ram           | 3476 
 sc_h          | 12   
 sc_w          | 7    
 talk_time     | 2    
 three_g       | 0    
 touch_screen  | 1    
 wifi          | 0    
only showing top 1 row



In [ ]:
df.describe().show()

+-------+-----------------+------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+
|summary|               id|     battery_power|               blue|       clock_speed|          dual_sim|               fc|            four_g|        int_memory|              m_dep|         mobile_wt|          n_cores|                pc|         px_height|          px_width|              ram|             sc_h|             sc_w|        talk_time|           three_g|     touch_screen|              wifi|
+-------+-----------------+------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+--------------

In [ ]:
# Считываем и объединяем данные (1 балл)
# Приведите все данные к правильному типу, либо считайе сразу верно (1 балл)
# Создаём колонки с соотношением сторон и плотностью пикселей (1 балл)
###############
df.select('sc_w').groupby('sc_w').count().sort('sc_w').show()
###############

+----+-----+
|sc_w|count|
+----+-----+
|   0|  112|
|   1|   99|
|  10|   52|
|  11|   29|
|  12|   21|
|  13|   23|
|  14|   20|
|  15|   15|
|  16|    7|
|  17|    8|
|  18|    5|
|   2|  107|
|   3|   84|
|   4|   95|
|   5|   93|
|   6|   73|
|   7|   67|
|   8|   44|
|   9|   46|
+----+-----+



## Сохранение


In [ ]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и временем разговора в формате "1day 1hour 1minute"
# 2 балла
###############
df.saveAsTable(hw_2, csv sep=';', encoding='cp1251')
###############

SyntaxError: ignored